# Stock data analysis

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
from urllib.parse import urlencode
import datetime

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [5]:
class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"

    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params = {"frequency": "1wk", "period1": datetime_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        html_table_data = soup.find("table")

        return title, html_table_data

In [6]:
def build_stock_analysis_prompt(stock_symbol, title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."
    
    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [7]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [8]:
display_analysis("GOOG")

## Analysis and Summary of GOOG Stock Trends

### Overview
The provided data outlines the weekly performance of Google's stock (GOOG) over the course of approximately one year. It includes various parameters like **Open**, **High**, **Low**, **Close**, **Adjusted Close**, and **Volume** for each week's trading, along with dividend information when applicable.

### Price Trends

1. **Overall Trend:**
   - **Price Rise:** The stock started at 143.43 on January 15, 2024, and ended at 189.72 on January 13, 2025. 
   - **Current Direction:** There has been a noticeable uptrend over the sampled months, indicating strong performance in the market.

2. **Price Fluctuation:**
   - **Highs and Lows:** Weekly highs reached up to 202.14 on January 6, 2025, and lows dropped to as low as 137.07 during the week of March 11, 2024. This underscores volatility in the stock's trading price.
   - **Recent Volatility:** There were significant price movements particularly in December 2024 and early January 2025.

3. **Closing Prices:**
   - The closing prices over the weeks show a substantial increase, especially in recent weeks when prices generally hovered around $190 after hitting lows of around $150 earlier in the year.

### Volume Trends
- **Trading Volume:** Weekly trading volumes indicate investor activity and interest levels in GOOG. For instance, the highest volume recorded was 156,317,400 on December 16, 2024, indicating heightened trading activity at that time.
- **Volume Consistency:** There are fluctuations in trading volume, which can indicate periods of uncertainty or strong interest from investors.

### Dividend Payments
- **Dividends Declared:** The data shows that GOOG issued dividends at least three times within the span, hinting at shareholder value returns.

### Total Returns Calculation

To calculate returns:

1. **Starting Price:** $143.43 (Jan 15, 2024)
2. **Ending Price:** $189.72 (Jan 13, 2025)

The formula for the percentage return is:

\[
\text{Percentage Return} = \left(\frac{\text{Ending Price} - \text{Starting Price}}{\text{Starting Price}}\right) \times 100
\]

Calculation:

\[
\text{Percentage Return} = \left(\frac{189.72 - 143.43}{143.43}\right) \times 100 = \left(\frac{46.29}{143.43}\right) \times 100 \approx 32.3\%
\]

### Conclusion 

- **Performance Summary:** GOOG has shown a positive upward trend throughout the year, with a total return of approximately **32.3%** over the period.
- **Investment Insights:** There have been periods of volatility, but the overall performance suggests that long-term holding may have yielded favorable returns for investors.
- **Monitoring Future Trends:** It’s critical to keep track of ongoing price movements and market conditions, especially with significant fluctuations and dividend announcements which could impact future investment decisions.

In [9]:
display_analysis("MSFT")

# MSFT Stock Analysis and Trends

Here's a summary of the weekly performance of Microsoft (MSFT) stock over the specified time period. The data shows various key metrics such as Opening Price, High Price, Low Price, Closing Price, Adjusted Closing Price, and Volume traded.

## Key Observations

### Price Movements:
1. **Opening and Closing Prices**: 
   - The stock opened at **415.23** on January 13, 2025, and closed at **398.67** on January 15, 2024. 
   - The highest recent opening price noted was **466.55** on July 8, 2024.
   - The stock fluctuated throughout the year, with closing prices reaching highs and lows due to market conditions.

2. **High and Low Prices**: 
   - During the analyzed period, the price reached a high of **467.70** and a low of **385.58**.
   - This reflects considerable volatility, typical for a high-tech stock like Microsoft.

3. **Overall Trend**: 
   - The stock experienced a general upward trend towards the middle of 2024, hitting its peak.
   - Thereafter, there were corrections that saw prices fall back, especially noted from Dec 2024 leading into early Jan 2025.

### Trading Volume:
- Fluctuations in trading volume were significant, with the highest recorded volume of **164,761,600** on January 29, 2024.
- This indicates varying interest and activity from traders, which can influence stock prices.

### Dividends:
- MSFT also paid several dividends throughout the year, with amounts such as **0.75** to **0.83** per share, reflecting the company's commitment to returning value to shareholders.

## Total Return Calculation

To calculate the total return over the period:
- **Initial Price (January 15, 2024)**: **398.67**
- **Final Price (January 13, 2025)**: **417.07**

### Calculation Formula:
\[
\text{Total Return} = \left( \frac{\text{Final Price} - \text{Initial Price}}{\text{Initial Price}} \right) \times 100\%
\]
\[
\text{Total Return} = \left( \frac{417.07 - 398.67}{398.67} \right) \times 100\% \approx 4.68\%
\]

### Conclusion
Investing in MSFT during this timeframe could have yielded approximately **4.68%** in total return, not accounting for dividends which would additionally increase this figure. The stock showed promising growth during the year, but with notable fluctuations that could present risks to investors. 

### Final Note:
For anyone interested in investing, it's crucial to consider both market trends and individual company performance, alongside broader economic indicators. Regular monitoring and staying updated on company news can help manage risks effectively.

In [10]:
display_analysis("AAPL")

# AAPL Stock Analysis Summary

Based on the provided weekly data for Apple Inc. (AAPL) stock, let’s analyze the trends over the past year.

## Key Observations:
1. **Price Trends**:
   - The stock began at **$182.16** on January 15, 2024, and reached **$231.40** by January 13, 2025.
   - The highest closing price during this period was **$255.59** recorded on December 23, 2024.
   - The lowest point was **$165.00** on April 15, 2024.

2. **Volatility**:
   - There were noticeable fluctuations in price throughout the year; for example, the stock jumped from **$185.85** to **$192.42** in a single week (January 29 to February 5, 2024).
   - The volume of shares traded also varied greatly, with peaks like **635,755,100** on June 10, 2024, indicating high interest or news that affected stock prices.

3. **Significant Events**:
   - Regular dividends were noted, such as **$0.25** per share on multiple occasions, which indicates that the stock not only grows but also provides returns via dividends.
   - The data suggests that the stock responded to market events, evident from the significant price changes following dividend announcements or earnings reports.

## Performance Calculation:
To calculate the total return percentage over the period:

1. **Initial Price (Jan 15, 2024)**: $182.16
2. **Final Price (Jan 13, 2025)**: $231.40

### Total Return Formula:
\[ 
\text{Total Return (\%)} = \left( \frac{\text{Final Price} - \text{Initial Price}}{\text{Initial Price}} \right) \times 100 
\]

Substituting in the values:

\[
\text{Total Return (\%)} = \left( \frac{231.40 - 182.16}{182.16} \right) \times 100 \approx 27.0\%
\]

## Conclusion:
Over this one-year period, AAPL showed a strong upward trend with a total return of approximately **27%**. This includes both price appreciation and dividends, making it a potentially rewarding investment for shareholders. However, the volatility of the stock suggests that while there are opportunities for gains, investors should also be aware of the risks associated with price fluctuations.

In [11]:
display_analysis("BIDU")

## BIDU Stock Analysis (Jan 2024 - Jan 2025)

### Summary of the Data
The data provided includes the weekly stock prices for BIDU (Baidu Inc) over a period of approximately one year. It covers the following key metrics:

1. **Open Price** - The price at which the stock opened for trading during the week.
2. **High Price** - The highest price reached during the week.
3. **Low Price** - The lowest price that the stock dropped to during the week.
4. **Close Price** - The final price at which the stock closed at the end of the week.
5. **Adjusted Close** - The closing price adjusted for any splits or dividends.
6. **Volume** - The number of shares traded during that week.

### General Trends

- **Starting and Ending Prices**
  - The starting price on January 15, 2024, was **$102.65**.
  - The closing price on January 13, 2025, was **$78.20**.
  
- **Price Movement**
  - Over the period, the stock experienced significant fluctuations.
  - Prices peaked at around **$113.44** on May 6, 2024, but then fell sharply to **$77.43** by the first week of January 2025.

- **Highs and Lows**
  - The stock saw a maximum weekly high of **$114.99** and consistent sell-offs leading to lower prices toward the end of the observation period.
  - The stock experienced significant downward pressure, especially noted by the dramatic fall from **$113.41** on April 29, 2024, to **$78.20** on January 13, 2025.

- **Volume of Trades**
  - Trading volumes were notably high during certain weeks, indicating increased investor activity, particularly around significant price movements.

### Performance Analysis
To find out the total returns, we can use the following formula:

\[
\text{Total Return (\%)} = \left( \frac{\text{Ending Price} - \text{Starting Price}}{\text{Starting Price}} \right) \times 100
\]

- **Starting Price (Jan 15, 2024)**: **$102.65**
- **Ending Price (Jan 13, 2025)**: **$78.20**

Now, applying the formula:

\[
\text{Total Return (\%)} = \left( \frac{78.20 - 102.65}{102.65} \right) \times 100
\]

\[
= \left( \frac{-24.45}{102.65} \right) \times 100 \approx -23.84\%
\]

### Conclusion
BIDU has seen a decline of approximately **23.84%** over the observed period. This drop indicates a downward trend, where investors might want to be cautious looking forward. Despite some fluctuations and high volumes, the general sentiment shows declining stock value through this timeframe. New investors should consider these trends and do further research before making financial decisions.